In [ ]:
# Installing
!pip install -q openai langchain gradio

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key")

Enter OpenAI API Key··········


In [4]:
from langchain.chat_models import ChatOpenAI

# Initialize chat model
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model_name="gpt-3.5-turbo",
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a prompt template
template = """You are an expert travel consultant. You give specific and thoughtful advice to potential travelers."""

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [6]:
from langchain.memory import ConversationBufferMemory

# Create conversation history store
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [7]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Compose components in a chain using LCEL
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | chat_prompt
    | llm
)

In [8]:
# Response function for ChatInterface
def stream_response(input, _):
    if input is not None:
        partial_message = ""
        for chunk in chain.stream({"input": input}): # LCEL streaming syntax
            partial_message += chunk.content
            yield partial_message

In [14]:
import gradio as gr


gr.ChatInterface(
    stream_response, retry_btn=None, undo_btn=None, clear_btn=None
).queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9756c5824adc686d1c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [15]:
# Prompt chaining example

from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate


# chain 1: Generating premises
comedian = "Kevin Hart"

template = """In bullet point form, wrie 5-10 potential topics for jokes about going to the movies that might be employed by {comedian}. Don't write the entire jokes just the premises. Mention the name of the comedian at the start of the bullet point\n"""
prompt_template = PromptTemplate(input_variables=["comedian"], template=template)
premises_chain = LLMChain(llm=llm, prompt=prompt_template)

# chain 2: Generate storyboard
template = """Here are a list of premises for jokes in the style of our requested comedian:
  {premises}
  From these bullet points, select 2-3 premises that could be combined into a funny joke, and note how they might flow with one and other.\n\n"""
prompt_template = PromptTemplate(input_variables=["premises"],template=template)
storyboard_chain = LLMChain(llm=llm, prompt=prompt_template)

# Chain 3: Create joke
template = """Using the following storyboard:
{storyboard}
  Please write a comedic joke in the style of our requested comedian.
  """
prompt_template = PromptTemplate(input_variables=["storyboard"],template=template)
final_joke_chain = LLMChain(llm=llm, prompt=prompt_template)
final_joke_chain_seq = SimpleSequentialChain(
      chains=[premises_chain, storyboard_chain, final_joke_chain],verbose=True
)

print(final_joke_chain_seq.run(comedian))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
- Kevin Hart: "The struggle of finding a seat at the movies when you're short"
- Kevin Hart: "When you take your kids to see an animated movie and end up enjoying it more than them"
- Kevin Hart: "The absurd prices of popcorn at the movies"
- Kevin Hart: "Trying to sneak snacks into the theater without getting caught"
- Kevin Hart: "The annoying person who talks throughout the entire movie"
- Kevin Hart: "The fear of accidentally sitting next to someone who talks to the screen"
- Kevin Hart: "The anxiety of choosing the right movie and ending up disappointed"
- Kevin Hart: "The frustration of sitting behind someone with a huge head at the movies"
- Kevin Hart: "The struggle of trying to hold in laughter during a funny movie"
- Kevin Hart: "The awkwardness of going to the movies alone and feeling judged"
Premises: 
- Kevin Hart: "The struggle of finding a seat at the movies when you're short"
- Kevin Hart: "The absurd prices of popcorn at 

In [ ]:
"""
Final exercise:

1) Like with our travel consultant above, we would like to create a chatbot.

Rather than a travel consultant, we would like this chatbot to be a financial consultant of which the user can ask personal finance questions.

Also, we would like to create a Gradio interface with streamed responses.


2) Create a prompt chain to write a poem about McDonald's in the style of a user supplied poet (similar to the comedian joke chain we created above).

"""

In [17]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# Define a prompt template
template = """You are an expert Financial consultant. You give specific and thoughtful advice on personal finance."""

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [18]:
from langchain.memory import ConversationBufferMemory

# Create conversation history store
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

In [19]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# Compose components in a chain using LCEL
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | chat_prompt
    | llm
)

In [20]:
# Response function for ChatInterface
def stream_response(input, _):
    if input is not None:
        partial_message = ""
        for chunk in chain.stream({"input": input}): # LCEL streaming syntax
            partial_message += chunk.content
            yield partial_message

In [21]:
gr.ChatInterface(
    stream_response, retry_btn=None, undo_btn=None, clear_btn=None
).queue().launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://45e7343ce4862174c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [22]:
# Prompt chaining example 2

from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate


# chain 1: Generating premises
poet = "Dante Alighieri"

template = """In bullet point form, wrie 5-10 potential topics for poems about McDonald employed by {poet}. Don't write the entire poem just the premises. Mention the name of the poet at the start of the bullet point\n"""
prompt_template = PromptTemplate(input_variables=["comedian"], template=template)
premises_chain = LLMChain(llm=llm, prompt=prompt_template)

# chain 2: Generate storyboard
template = """Here are a list of premises for poems in the style of our requested poet:
  {premises}
  From these bullet points, select 2-3 premises that could be combined into a great poem, and note how they might flow with one and other.\n\n"""
prompt_template = PromptTemplate(input_variables=["premises"],template=template)
storyboard_chain = LLMChain(llm=llm, prompt=prompt_template)

# Chain 3: Create joke
template = """Using the following storyboard:
{storyboard}
  Please write a great poem in the style of our requested poet.
  """
prompt_template = PromptTemplate(input_variables=["storyboard"],template=template)
final_joke_chain = LLMChain(llm=llm, prompt=prompt_template)
final_joke_chain_seq = SimpleSequentialChain(
      chains=[premises_chain, storyboard_chain, final_joke_chain],verbose=True
)

print(final_joke_chain_seq.run(poet))



> Entering new SimpleSequentialChain chain...
Dante Alighieri:

- The paradox of fast food: Exploring the contrast between McDonald's quick service and Dante's renowned masterpiece, "The Divine Comedy."
- The journey of a Big Mac: Imagining the odyssey of a McDonald's burger from its creation to its consumption, drawing parallels to Dante's journey through Hell, Purgatory, and Paradise.
- The infernal experience of working at McDonald's: Depicting the struggles and challenges faced by employees, reminiscent of Dante's depiction of the torments of Hell.
- The temptation of the golden arches: Reflecting on the allure and addictive nature of McDonald's food, drawing inspiration from Dante's exploration of sin and temptation.
- The circle of drive-thru: Portraying the repetitive nature of working at the drive-thru, akin to Dante's portrayal of the souls trapped in eternal circles in Hell.
- McFlurry of emotions: Exploring the emotional roller coaster experienced by McDonald's employees, 